In [1]:
# Import libraries
import numpy as np
import pandas as pd
from sklearn.neighbors import NearestNeighbors
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors

In [2]:
#path data base
path_movies = '/Users/claudiojunior/Desktop/Python/Data_base/movies.csv'
path_ratings = '/Users/claudiojunior/Desktop/Python/Data_base/ratings.csv'

In [3]:
#creating Data Frame
df_movies = pd.DataFrame(pd.read_csv(path_movies))
df_ratings = pd.DataFrame(pd.read_csv(path_ratings))

In [4]:
df_movies

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
34203,151697,Grand Slam (1967),Thriller
34204,151701,Bloodmoney (2010),(no genres listed)
34205,151703,The Butterfly Circus (2009),Drama
34206,151709,Zero (2015),Drama|Sci-Fi


In [5]:
df_ratings

,userId,movieId,rating,timestamp
0,1,169,2.5,1204927694
1,1,2471,3.0,1204927438
2,1,48516,5.0,1204927435
3,2,2571,3.5,1436165433
4,2,109487,4.0,1436165496
...,...,...,...,...
22884372,247753,49530,5.0,1430437962
22884373,247753,69481,3.0,1430437984
22884374,247753,74458,4.0,1430437968
22884375,247753,76093,5.0,1430437811


In [6]:
#Quantity of ratings per user
df_ratings['userId'].value_counts()

185430    9281
46750     7515
204165    7057
135877    6015
58040     5801
          ... 
127573       1
180933       1
200783       1
180936       1
154112       1
Name: userId, Length: 247753, dtype: int64

In [7]:
#looking for users that had rating for more than 200 movies
x = df_ratings['userId'].value_counts() > 200

In [8]:
x

185430     True
46750      True
204165     True
135877     True
58040      True
          ...  
127573    False
180933    False
200783    False
180936    False
154112    False
Name: userId, Length: 247753, dtype: bool

In [9]:
#Quantity of users
#user_ids
y = x[x].index
print(y.shape)

(28729,)


In [10]:
y

Int64Index([185430,  46750, 204165, 135877,  58040,  94166,  16382, 246533,
             49237, 239662,
            ...
             45229, 208217, 207284, 245695,   1916, 215895,  15268,  92689,
            240970,  41540],
           dtype='int64', length=28729)

In [11]:
#Filter using an inner join method, we will bring just users that gave more than 200 ratings
ratings = df_ratings[df_ratings['userId'].isin(y)]

In [12]:
ratings

,userId,movieId,rating,timestamp
749,15,1,4.0,1137894004
750,15,7,3.0,1153012997
751,15,17,3.5,1293228198
752,15,24,4.0,1153013415
753,15,32,4.5,1137893119
...,...,...,...,...
22883548,247735,8961,4.5,1101128800
22883549,247735,8983,3.0,1105547149
22883550,247735,27773,4.5,1113555826
22883551,247735,31878,4.0,1118225293


In [13]:
#merge by movieiD and creating a new DF name rating_with_movies
rating_with_movies = ratings.merge(df_movies, on = 'movieId')
rating_with_movies.head()

,userId,movieId,rating,timestamp,title,genres
0,15,1,4.0,1137894004,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,17,1,5.0,944991567,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,37,1,4.5,1227401937,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
3,47,1,4.0,1150032907,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
4,50,1,4.0,951519605,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy


In [14]:
#Analyzing how many movies received ratings and doing a count using groupby method
number_rating = rating_with_movies.groupby('title')['rating'].count().reset_index()
number_rating

,title,rating
0,"""Great Performances"" Cats (1998)",49
1,#1 Cheerleader Camp (2010),3
2,#chicagoGirl: The Social Network Takes on a Di...,2
3,$ (Dollars) (1971),11
4,$5 a Day (2008),18
...,...,...
33055,Škola princů (2010),1
33056,أهواك (2015),1
33057,キサラギ (2007),1
33058,ドラゴンボール Ｚ あつまれ！ 悟空ワールド (1992),2


In [15]:
#rename the columns rating to number of rating, just because is a count
number_rating.rename(columns ={'rating':'number_of_ratings'}, inplace = True)
number_rating

,title,number_of_ratings
0,"""Great Performances"" Cats (1998)",49
1,#1 Cheerleader Camp (2010),3
2,#chicagoGirl: The Social Network Takes on a Di...,2
3,$ (Dollars) (1971),11
4,$5 a Day (2008),18
...,...,...
33055,Škola princů (2010),1
33056,أهواك (2015),1
33057,キサラギ (2007),1
33058,ドラゴンボール Ｚ あつまれ！ 悟空ワールド (1992),2


In [16]:
#Merge of the two Data Frames by title, just to add the quantity of rating og each movie
final_rating = rating_with_movies.merge(number_rating, on ='title')
final_rating

,userId,movieId,rating,timestamp,title,genres,number_of_ratings
0,15,1,4.0,1137894004,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,20504
1,17,1,5.0,944991567,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,20504
2,37,1,4.5,1227401937,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,20504
3,47,1,4.0,1150032907,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,20504
4,50,1,4.0,951519605,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,20504
...,...,...,...,...,...,...,...
13524069,247081,117561,3.5,1417076301,Once Upon a Time in Queens (2013),Comedy|Drama,1
13524070,247081,125309,3.0,1441977025,Winnie Mandela (2013),Drama,1
13524071,247509,95581,4.0,1341630851,"Flying Fleet, The (1929)",Adventure|Drama|Romance,1
13524072,247534,111838,4.0,1429019913,"Man Named Rocca, A (Nommé La Rocca, Un) (Man C...",Drama,1


In [17]:
#By business decision we will just look to movies with more than 50 ratings
final_rating = final_rating[final_rating['number_of_ratings'] >= 50]

In [18]:
#to avoid distorcions we must drop the duplicates lines
final_rating.drop_duplicates(['userId','title'],inplace = True)
final_rating.shape

/var/folders/9j/zrp1cqxs43zgd02tvg6m6m740000gp/T/ipykernel_1425/1001651081.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_rating.drop_duplicates(['userId','title'],inplace = True)


(13332453, 7)

In [19]:
final_rating

,userId,movieId,rating,timestamp,title,genres,number_of_ratings
0,15,1,4.0,1137894004,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,20504
1,17,1,5.0,944991567,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,20504
2,37,1,4.5,1227401937,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,20504
3,47,1,4.0,1150032907,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,20504
4,50,1,4.0,951519605,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,20504
...,...,...,...,...,...,...,...
13483896,229481,2246,3.0,940729849,Stars and Bars (1988),Action|Comedy|Romance,54
13483897,231981,2246,2.0,962657557,Stars and Bars (1988),Action|Comedy|Romance,54
13483898,232411,2246,3.0,942068959,Stars and Bars (1988),Action|Comedy|Romance,54
13483899,235888,2246,3.0,1220228010,Stars and Bars (1988),Action|Comedy|Romance,54


In [20]:
#to use the sparce matrix we must create a specie of matrix doing a pivot of turn possible apply the fuction os sparce matrix
movie_pivot = final_rating.pivot_table(columns = 'userId', index = 'title', values = 'rating')
movie_pivot

userId,15,17,20,37,38,39,47,50,59,68,...,247624,247662,247700,247702,247704,247705,247725,247730,247732,247735
title,,,,,,,,,,,,,,,,,,,,,
'71 (2014),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
'Round Midnight (1986),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
'Salem's Lot (2004),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
'Til There Was You (1997),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"'burbs, The (1989)",NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,3.0,NaN,4.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
loudQUIETloud: A Film About the Pixies (2006),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
xXx (2002),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0
xXx: State of the Union (2005),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


We will use Nearest Neighbors to create a movie suggestion, so for that we will calculate the distance between max and min value, considering that our matrix have a lot of Nan we must at first change it to zero using fillna function and after that change it to sparce matrix, the only reason to change to sparce matrix is that we reduce the computacional cost comparting to the tradicional matrix.

In [21]:
#Replace NaN for o
movie_pivot.fillna(0, inplace = True)

Matriz esparca, quando temos muito valores zero o poder de computação aumentará para calcular a distancia dos valores, sendo assim, convertemos a tabela dinâmica para matriz esparsa e em seguida alimentaremos o modelo

In [22]:
#Creating the sparce matrix
movie_sparse = csr_matrix(movie_pivot)
movie_sparse

<10718x28729 sparse matrix of type '<class 'numpy.float64'>'
	with 13332453 stored elements in Compressed Sparse Row format>

In [23]:
#Using Neatrst Neighbors to fit our algorithm
model=NearestNeighbors(algorithm='brute')
model.fit(movie_sparse)

NearestNeighbors(algorithm='brute')

In [26]:
#creating a function that will bring an suggestion according to the index of the movie at Data Frame
def recomendation(label):
    distances,suggestions = model.kneighbors(movie_pivot.iloc[label,:].values.reshape(1,-1))
    for i in range(len(suggestions)):
        movie_pivot.index[suggestions[i]]
    return print('You may like:' f"{movie_pivot.index[suggestions[i]]}")

In [30]:
label = 250
recomendation(label)

You may like:Index(['Adventures of Sharkboy and Lavagirl 3-D, The (2005)',
       'SuperBabies: Baby Geniuses 2 (2004)',
       'Giant Spider Invasion, The (1975)', 'Doogal (2006)',
       'Horrors of Spider Island (Ein Toter Hing im Netz) (1960)'],
      dtype='object', name='title')
